# Router Engine

Given a query, the router will pick what to run.  This shows dynamic query understanding.

## Setup

In [1]:
import os
import dotenv

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)


True

In [1]:
# this is required since Juypyter notebook is not async by default.
import nest_asyncio
nest_asyncio.apply()

## Load data

In [4]:
from llama_index.core import SimpleDirectoryReader

# load documents

lightning_ai="/teamspace/studios/this_studio/retrieval-augmented-generation/data/transcript.pdf"
github_codespace="./data/transcript.pdf"
documents = SimpleDirectoryReader(input_files=[lightning_ai]).load_data()


## Split document into nodes

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

## Define LLM and Embedding Model

In [6]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Indexes

- Summary Index 
- Vector Index 

Think of the index as a set of metadata over the data.  Different indexes have different index behavior.  A vector index indexes node through text embeddings.  Querying a vector index in llamaindex will return the most similar nodes. A summary index is also a simple index, and a query to it will return all of the nodes that are currently in the index.

In [7]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Turn the indexes into query engines

Each query engine represents an interface over the data that has been stored in the index.  It combines retrieval with LLM synthesis.  Each engine is specific for a certain kind of question.  This is where the use case of the router becomes significant.  

In [8]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

vector_query_engine = vector_index.as_query_engine()

## Create query tools

The query tools are simply the query engine with additional metadata.  The metadata reflects the kind of information the tool can answer.  For example, asking it to "summarize" the article would launch the summary_tool.

In [9]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the document."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the document."
    ),
)

## Define Router

Llamaindex provides several diffent selectors.  

- The LLM selectors use the LLM to output a JSON that is parsed, and corresponding indexes are queried.
- The Pydantic selects use the OpenAI Function Calling API to produce pydantic selection objects rather than raw JSON.

In this example we are using the LLMSingleSelector. The RouterQueryEngine takes in a selector type as well as a set of tools.

In [10]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    # allows us to view the intermediate steps of the query engine
    verbose=True,
)

In [11]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to the document..
The document discusses the importance of bridging the Product Company Gap, emphasizing the need to move beyond just product Market fit. It highlights the significance of building a product that has a good value proposition, targets a minimum viable segment, and focuses on repeatable success. The discussion also covers aspects like designing for product go-to-market fit, creating instant and ongoing value for customers, and playing well in the ecosystem through partnerships. Additionally, it touches on pricing strategies, simplifying product installation, and leveraging partnerships to accelerate business growth. Overall, the document provides insights on how to develop a product that can scale into a successful company.


In [12]:
print(len(response.source_nodes))

37


In [13]:
response = query_engine.query("How do you build a product according to this document?")
print(str(response))

Selecting query engine 1: Useful for retrieving specific context from the document..
To build a product according to this document, you should start by designing a product for go-to-market fit, ensuring that you create a minimum viable product (MVP) that addresses a valuable problem. It is crucial to triple check the value proposition before investing in hiring engineers or spending money to ensure that the product solves a problem that customers are willing to pay for. Additionally, it is important to design the product with scalability in mind, considering factors like go-to-market strategy, pricing, and building a business model that supports scaling the product into a successful company.


## Let's put everything together

Code from the above cells, have been put into the utils helper method.  Builds the query engine with both vector and summary search.

In [14]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("./data/transcript.pdf")

In [15]:
response = query_engine.query("Tell me about the document.")
print(str(response))

Selecting query engine 1: The choice is more relevant as it specifically mentions retrieving specific context from the MetaGPT paper..
The document appears to be a transcript from a discussion or presentation focused on product development and design principles. It includes conversations about simplicity in product design, the importance of focusing on solving basic problems easily, and the value of creating products that address specific needs. The discussion also touches on the concept of minimum viable segments, the importance of proving value early on, and strategies for streamlining time to value for different types of companies.


In [16]:
response = query_engine.query("Does the paper provide specific coding example?")
print(str(response))

Selecting query engine 1: The paper provides specific context from the MetaGPT paper, which may include coding examples..
The paper does not provide specific coding examples.
